In [1]:
%run 'dataprocessing.ipynb'

Loading training set...
Done
Loading test set...
Done
Splitting into val/test set...
Done
Normalization...
Done
Converting to one-hot code...
Done


In [2]:
def add_gaussian_noise(images):
    gaussian_noise_imgs = []
    row, col, _ = images[0].shape
    # Gaussian distribution parameters
    mean = 0
    var = 0.1
    sigma = var ** 0.5   
    for image in images:
        gaussian = np.random.random((row, col, 1)).astype(np.float64)
        gaussian = np.concatenate((gaussian, gaussian, gaussian), axis = 2)
        gaussian_img = cv2.addWeighted(image, 0.75, gaussian, 0.25, 0)
        gaussian_noise_imgs.append(gaussian_img)
    gaussian_noise_imgs = np.array(gaussian_noise_imgs, dtype = np.float32)
    return gaussian_noise_imgs

In [3]:
print("Generating Gaussian noise images...")

train_gaussian_images = add_gaussian_noise(train_images)

print("Done")

Generating Gaussian noise images...
Done


In [4]:
print("Stacking Gaussian noise images...")

X_train=np.vstack((train_images,train_gaussian_images))
Y_train=np.vstack((train_labels,train_labels))

print("Done")

Stacking Gaussian noise images...
Done


def add_salt_pepper_noise(X_imgs):
    # Need to produce a copy as to not modify the original image
    X_imgs_copy = X_imgs.copy()
    row, col, _ = X_imgs_copy[0].shape
    salt_vs_pepper = 0.2
    amount = 0.004
    num_salt = np.ceil(amount * X_imgs_copy[0].size * salt_vs_pepper)
    num_pepper = np.ceil(amount * X_imgs_copy[0].size * (1.0 - salt_vs_pepper))
    
    for X_img in X_imgs_copy:
        # Add Salt noise
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in X_img.shape]
        X_img[coords[0], coords[1], :] = 1

        # Add Pepper noise
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in X_img.shape]
        X_img[coords[0], coords[1], :] = 0
    return X_imgs_copy

print("Generating pepper noise images...")

train_pepper_images = add_salt_pepper_noise(train_images)

print("Done")

print("Stacking pepper noise images...")

X_train=np.vstack((X_train,train_pepper_images))
Y_train=np.vstack((Y_train,train_labels))

print("Done")

In [5]:
def get_mask_coord(imshape):
    vertices = np.array([[(0.09 * imshape[1], 0.99 * imshape[0]), 
                          (0.43 * imshape[1], 0.32 * imshape[0]), 
                          (0.56 * imshape[1], 0.32 * imshape[0]),
                          (0.85 * imshape[1], 0.99 * imshape[0])]], dtype = np.int32)
    return vertices

def get_perspective_matrices(X_img):
    offset = 15
    img_size = (X_img.shape[1], X_img.shape[0])

    # Estimate the coordinates of object of interest inside the image.
    src = np.float32(get_mask_coord(X_img.shape))
    dst = np.float32([[offset, img_size[1]], [offset, 0], [img_size[0] - offset, 0], 
                      [img_size[0] - offset, img_size[1]]])
    
    perspective_matrix = cv2.getPerspectiveTransform(src, dst)
    return perspective_matrix

def perspective_transform(X_img):
    # Doing only for one type of example
    perspective_matrix = get_perspective_matrices(X_img)
    warped_img = cv2.warpPerspective(X_img, perspective_matrix,
                                     (X_img.shape[1], X_img.shape[0]),
                                     flags = cv2.INTER_LINEAR)
    return warped_img

In [6]:
print("Generating perspective images...")

train_perspective_images=[]
for image in train_images:
    perspective_image = perspective_transform(image)
    train_perspective_images.append(perspective_image)
    
train_perspective_images = np.array(train_perspective_images, dtype = np.float32)    

print("Done")

In [8]:
print("Stacking perspective images...")

X_train=np.vstack((X_train,train_perspective_images))
Y_train=np.vstack((Y_train,train_labels))

print("Done")

In [9]:
X_val=val_images
Y_val=val_labels
X_test=test_images
Y_test=test_labels

In [ ]:
print("Finished!")